### Importar Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

### Importar Arquivos

#### Candidatos

In [2]:
candidatos = pd.read_csv('~/Dados/consulta_cand_2024/consulta_cand_2024_BRASIL.csv',sep=';',encoding='latin')

In [ ]:
candidatos.head()

In [ ]:
candidatos.info(verbose=True)

In [5]:
for col in [col for col in candidatos.columns if col.startswith('CD_')]:
    if col.startswith('CD_'):
        candidatos.drop(col, axis=1, inplace=True)


In [ ]:
candidatos.info(verbose=True)

In [ ]:
for col in [col for col in candidatos.columns if col.startswith('DT_')]:
    candidatos[col] = pd.to_datetime(candidatos[col], format='%d/%m/%Y', errors='coerce')


candidatos.info(verbose=True)


In [ ]:
candidatos.head()

In [ ]:
candidatos.isna().sum()

In [10]:
candidatos.dropna(inplace=True)

#### Complemento

In [11]:
complementos = pd.read_csv('~/Dados/consulta_cand_complementar_2024/consulta_cand_complementar_2024_BRASIL.csv',sep=';',encoding='latin')

In [ ]:
complementos.head()

In [13]:
for col in [col for col in complementos.columns if col.startswith('CD_')]:
    complementos.drop(col, axis=1, inplace=True)


In [14]:
for col in [col for col in complementos.columns if col.startswith('DT_')]:
    complementos[col] = pd.to_datetime(complementos[col], format='%d/%m/%Y', errors='coerce')


In [ ]:
complementos.head()

#### Bens

In [16]:
bens = pd.read_csv('~/Dados/bem_candidato_2024/bem_candidato_2024_BRASIL.csv',sep=';',encoding='latin')

In [ ]:
bens.head()

In [18]:
for col in [col for col in bens.columns if col.startswith('CD_')]:
    bens.drop(col, axis=1, inplace=True)


In [19]:
for col in [col for col in bens.columns if col.startswith('DT_')]:
    bens[col] = pd.to_datetime(bens[col], format='%d/%m/%Y', errors='coerce')


In [ ]:
bens.head()

In [ ]:
bens.info(verbose=True)

In [ ]:
bens['VR_BEM_CANDIDATO'] = pd.to_numeric(bens['VR_BEM_CANDIDATO'].str.replace(',', '.'))

bens['VR_BEM_CANDIDATO']
#bens['VR_BEM_CANDIDATO'] = bens['VR_BEM_CANDIDATO'].astype(float)

In [ ]:
bens['VR_BEM_CANDIDATO'].describe()

#### EDA

In [ ]:
candidatos.head()

In [ ]:
dados = candidatos.merge(complementos, how='left', on='SQ_CANDIDATO')
dados = dados.merge(bens,how='left',on='SQ_CANDIDATO')

dados.head()

In [26]:
#lista = dados.filter(regex='_x$').columns
#dados.drop(lista, axis=1, inplace=True)


In [27]:
#lista = dados.filter(regex='_y$').columns
#dados.drop(lista, axis=1, inplace=True)


In [ ]:
dados.sort_values(by=['SQ_CANDIDATO','NR_ORDEM_BEM_CANDIDATO']).head(10)

In [ ]:
dados.columns

In [ ]:
dados = dados[dados['VR_BEM_CANDIDATO'] < 30000000]

dados = dados[dados['NR_IDADE_DATA_POSSE'] < 130]

#dados = dados[dados['SG_UF_x'].isin(['MG','SP','RJ','ES'])]

dados['VR_BEM_CANDIDATO'].describe()

In [ ]:
dados['NR_IDADE_DATA_POSSE'].describe()

In [ ]:
dados_agg = dados.groupby(
    [  'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'SG_PARTIDO', 'SG_UF_x',
       'NM_PARTIDO', 'SG_UF_NASCIMENTO', 'DT_NASCIMENTO',
       'DS_GENERO', 'DS_GRAU_INSTRUCAO',
       'DS_ESTADO_CIVIL', 'DS_COR_RACA', 'DS_OCUPACAO',
       'NR_IDADE_DATA_POSSE', 'ST_QUILOMBOLA',
       'DS_ETNIA_INDIGENA', 'VR_DESPESA_MAX_CAMPANHA', 'ST_REELEICAO']
).agg(
    TOTAL_BENS=('VR_BEM_CANDIDATO', 'sum'),     # Nome personalizado para soma
        QUANT_BENS=('VR_BEM_CANDIDATO', 'count') # Nome personalizado para contagem
).reset_index()


dados_agg.head()

### EDA

In [ ]:
print(dados_agg.isnull().sum())  # Contar os valores nulos em cada coluna


In [ ]:
print(dados_agg.duplicated().sum())

In [ ]:
dados_agg.rename(columns={'DS_CARGO':'Cargo', 
                            'SQ_CANDIDATO':'SQ_CANDIDATO', 
                            'NR_CANDIDATO':'Num Candidato', 
                            'NM_CANDIDATO':'Nome Candidato',
                            'SG_PARTIDO':'Sigla Partido', 
                            'SG_UF_x': 'UF', 
                            'NM_PARTIDO':'Nome Partido', 
                            'SG_UF_NASCIMENTO':'UF Nascimento',
                            'DT_NASCIMENTO':'df Nascimento', 
                            'DS_GENERO':'Genêro', 
                            'DS_GRAU_INSTRUCAO':'Grau Instrução', 
                            'DS_ESTADO_CIVIL':'Estado Civil',
                            'DS_COR_RACA':'Cor / Raça', 
                            'DS_OCUPACAO':'Ocupação', 
                            'NR_IDADE_df_POSSE':'Idade', 
                            'ST_QUILOMBOLA':'Quilombola',
                            'DS_ETNIA_INDIGENA':'Etnia Indigina', 
                            'VR_DESPESA_MAX_CAMPANHA':'Despesa de Campanha', 
                            'ST_REELEICAO':'Releição',
                            'TOTAL_BENS': 'Valor Bens', 
                            'QUANT_BENS': 'Quant Bens',
                            'NR_IDADE_DATA_POSSE':'Idade'}, inplace=True)
dados_agg.head()

In [ ]:
print(dados_agg.columns)  # Lista os nomes das colunas


### Análises

![Gráficos](/Users/marcellofelipelli/Documents/dfAnalyticsPortfolio/LearningProcess/image.png)


#### Funcoes para Graficos

In [37]:
import plotly.express as px

def create_bar_chart(df, x_column, y_column, chart_title, x_axis_title, y_axis_title):
    """
    Função para criar um gráfico de barras com Plotly Express.
    
    Parâmetros:
    - df: dfFrame com os dados.
    - x_column: Coluna para o eixo X.
    - y_column: Coluna para o eixo Y.
    - chart_title: Título do gráfico.
    - x_axis_title: Título do eixo X.
    - y_axis_title: Título do eixo Y.
    """
    # Criar gráfico de barras
    fig = px.bar(
        df, 
        color=x_column, 
        x=x_column,  # Eixo X
        y=y_column,  # Eixo Y
        title=chart_title  # Título do gráfico
    )

    # Atualizar os títulos dos eixos
    fig.update_layout(
        xaxis_title=x_axis_title,  # Título do eixo X
        yaxis_title=y_axis_title,  # Título do eixo Y
        width=900,  # Largura do gráfico
        height=400  # Altura do gráfico
    )

    # Mostrar os valores nas barras
    fig.update_traces(texttemplate='%{y:,.2f}', textposition='inside')

    # Exibir o gráfico
    fig.show()

# Exemplo de uso
# create_bar_chart(data, 'SG_UF_NASCIMENTO', 'proportion', 
#                  'Candidatos: Valor de Bens > Valor Máximo Despesa', 
#                  'Estado de Nascimento', 'Quantidade de Candidatos')


In [38]:
import plotly.express as px
import pandas as pd

def create_countplot_plotly(data, x_column, chart_title, x_axis_title, y_axis_title):
    """
    Função para criar um gráfico de contagem (countplot) com Plotly Express.
    
    Parâmetros:
    - data: DataFrame com os dados.
    - x_column: Coluna para o eixo X (dados categóricos).
    - chart_title: Título do gráfico.
    - x_axis_title: Título do eixo X.
    - y_axis_title: Título do eixo Y.
    """
    # Contagem de valores da coluna categórica
    count_data = data[x_column].value_counts().reset_index()
    count_data.columns = [x_column, 'count']

    # Criar gráfico de barras para a contagem
    fig = px.bar(
        count_data, 
        color=x_column,
        x=x_column,  # Eixo X
        y='count',  # Eixo Y
        title=chart_title  # Título do gráfico
    )

    # Atualizar os títulos dos eixos
    fig.update_layout(
        xaxis_title=x_axis_title,  # Título do eixo X
        yaxis_title=y_axis_title,  # Título do eixo Y
        width=900,  # Largura do gráfico
        height=400  # Altura do gráfico
    )

    # Mostrar os valores nas barras
    fig.update_traces(texttemplate='%{y}', textposition='inside')

    # Exibir o gráfico
    fig.show()

# Exemplo de uso
# create_countplot_plotly(data, 'SG_UF_NASCIMENTO', 
#                         'Candidatos por Estado de Nascimento', 
#                         'Estado de Nascimento', 'Quantidade de Candidatos')


In [39]:
import plotly.express as px

def create_boxplot_plotly(data, x_column, y_column, chart_title, x_axis_title, y_axis_title):
    """
    Função para criar um gráfico de boxplot com Plotly Express.
    
    Parâmetros:
    - data: DataFrame com os dados.
    - x_column: Coluna para o eixo X (pode ser categórica).
    - y_column: Coluna para o eixo Y (numérica).
    - chart_title: Título do gráfico.
    - x_axis_title: Título do eixo X.
    - y_axis_title: Título do eixo Y.
    """
    # Criar boxplot
    fig = px.box(
        data, 
        color=x_column,
        x=x_column,  # Eixo X (normalmente categórico)
        y=y_column,  # Eixo Y (dados numéricos)
        title=chart_title  # Título do gráfico
    )

    # Atualizar os títulos dos eixos
    fig.update_layout(
        xaxis_title=x_axis_title,  # Título do eixo X
        yaxis_title=y_axis_title,  # Título do eixo Y
        width=900,  # Largura do gráfico
        height=400  # Altura do gráfico
    )

    # Exibir o gráfico
    fig.show()

# Exemplo de uso
# create_boxplot_plotly(data, 'SG_UF_NASCIMENTO', 'proportion', 
#                       'Boxplot de Proporção por Estado de Nascimento', 
#                       'Estado de Nascimento', 'Proporção')


In [40]:
import plotly.express as px

def create_distplot_plotly(data, column, chart_title, x_axis_title, y_axis_title, show_density=True):
    """
    Função para criar um gráfico de distribuição (similar ao distplot) com Plotly Express.
    
    Parâmetros:
    - data: DataFrame com os dados.
    - column: Coluna numérica para criar o gráfico de distribuição.
    - chart_title: Título do gráfico.
    - x_axis_title: Título do eixo X.
    - y_axis_title: Título do eixo Y.
    - show_density: Booleano para mostrar ou não a curva de densidade.
    """
    # Criar histograma com ou sem curva de densidade
    fig = px.histogram(
        data, 
        x=column, 
        nbins=30,  # Número de bins (intervalos)
        histnorm='probability density' if show_density else None,  # Para normalizar como densidade
        title=chart_title
    )

    # Atualizar os títulos dos eixos
    fig.update_layout(
        xaxis_title=x_axis_title,  # Título do eixo X
        yaxis_title=y_axis_title,  # Título do eixo Y
        width=900,  # Largura do gráfico
        height=400  # Altura do gráfico
    )

    # Mostrar o gráfico
    fig.show()

# Exemplo de uso
# create_distplot_plotly(df, 'proportion', 
#                        'Distribuição de Proporção', 
#                        'Proporção', 'Densidade de Probabilidade')


#### Candidatos com Valor de Bens maior que a Despesa Máxima para Campanha

In [ ]:
data = dados_agg[dados_agg['Despesa de Campanha']<dados_agg['Valor Bens']]

data.head()


In [ ]:
data.shape

In [ ]:
estados = pd.DataFrame(data['UF'].value_counts()).head(10).index.tolist()
estados

##### UF Nascimento

In [ ]:
df = data['UF Nascimento'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

df = df[df['UF Nascimento'].isin(estados)]

create_bar_chart(df, 'UF Nascimento', 'proportion', 
                 'Candidatos por Origem (UF): Valor de Bens > Valor Máximo Despesa (em %)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


df_geral = dados_agg[dados_agg['UF Nascimento'].isin(estados)]['UF Nascimento'].value_counts(normalize=True).reset_index()

df_geral['proportion'] = df_geral['proportion']*100

create_bar_chart(df_geral, 'UF Nascimento', 'proportion', 
                 'Candidatos por Origem (UF): Todos os Candidatos (em %)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')




In [ ]:
df = data[['Valor Bens','UF Nascimento']]

df = df[df['UF Nascimento'].isin(estados)]

#df = df.groupby('UF Nascimento')['Valor Bens'].sum().reset_index()

df = df.sort_values('Valor Bens',ascending=False)

df.head()

create_bar_chart(df.groupby('UF Nascimento')['Valor Bens'].sum().reset_index().reset_index().sort_values('Valor Bens',ascending=False), 'UF Nascimento', 'Valor Bens', 
                'Candidatos: Valor de Bens > Valor Máximo Despesa (em R$)', 
                'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:

df_geral = dados_agg[['Valor Bens','UF Nascimento']]

df_geral = df_geral[df_geral['UF Nascimento'].isin(estados)]

#df = df.groupby('UF Nascimento')['Valor Bens'].sum().reset_index()

df_geral = df_geral.sort_values('Valor Bens',ascending=False)

create_bar_chart(df_geral.groupby('UF Nascimento')['Valor Bens'].sum().reset_index().reset_index().sort_values('Valor Bens',ascending=False), 'UF Nascimento', 'Valor Bens', 
                 'Candidatos por Origem (UF): Todos os Candidatos (em R$)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
for estado in estados:
    create_boxplot_plotly(df[df['UF Nascimento']==estado], 'UF Nascimento', 'Valor Bens', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')
    


In [ ]:
df = data['UF Nascimento'].value_counts(normalize=True).reset_index()


df = df[df['UF Nascimento'].isin(estados)]

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'UF Nascimento', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa (em %)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg[dados_agg['Despesa de Campanha']<dados_agg['Valor Bens']]['Genêro'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'Genêro', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg[['Valor Bens','Genêro']]

create_boxplot_plotly(df, 'Genêro', 'Valor Bens', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')



In [ ]:
df = dados_agg['Genêro'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'Genêro', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg[dados_agg['Despesa de Campanha']<dados_agg['Valor Bens']]['Grau Instrução'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'Grau Instrução', 'proportion', 
                 'Candidatos: Por Grau Instrução', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg['Grau Instrução'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Grau Instrução', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg[['Sigla Partido']].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

df.head()

create_bar_chart(df, 'Sigla Partido', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg['Sigla Partido'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Sigla Partido', 'proportion', 
                 'Candidatos: Por Partido', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg[dados_agg['Despesa de Campanha']<dados_agg['Valor Bens']]['Estado Civil'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Estado Civil', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg['Estado Civil'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Estado Civil', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['Cor / Raça'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Cor / Raça', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg['Cor / Raça'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Cor / Raça', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['Ocupação'].value_counts(normalize=True).reset_index().head(10)

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Ocupação', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['Ocupação'].value_counts(normalize=True).reset_index().head(10)

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Ocupação', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['UF'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'UF', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = data['UF'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

df = df[df['UF'].isin(estados)]

create_bar_chart(df, 'UF', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = data[data['Etnia Indigina'] != '#NULO#']

df = df[df['Etnia Indigina'] != 'NÃO INFORMADO']

df = df['Etnia Indigina'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df.head(15),
                 'Etnia Indigina',
                 'proportion',
                 'Candidatos: Valor de Bens > Valor Máximo Despesa',
                 "Etnia Indígena",
                 "Quantidade de Candidatos")


In [ ]:
df = dados_agg[dados_agg['Etnia Indigina'] != '#NULO#']

df = df[df['Etnia Indigina'] != 'NÃO INFORMADO']

df = df['Etnia Indigina'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df.head(15),
                 'Etnia Indigina',
                 'proportion',
                 'Candidatos: Valor de Bens > Valor Máximo Despesa',
                 "Etnia Indígena",
                 "Quantidade de Candidatos")


#### Candidatos com Valor de Bens menor que a Despesa Máxima para Campanha

In [ ]:
data = dados_agg[dados_agg['Despesa de Campanha']>=dados_agg['Valor Bens']]

data.head()


In [ ]:
data.shape

In [ ]:
estados = pd.DataFrame(data['UF'].value_counts()).head(10).index.tolist()
estados

##### UF Nascimento

In [ ]:
df = data['UF Nascimento'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

df = df[df['UF Nascimento'].isin(estados)]

create_bar_chart(df, 'UF Nascimento', 'proportion', 
                 'Candidatos por Origem (UF): Valor de Bens > Valor Máximo Despesa (em %)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


df_geral = dados_agg[dados_agg['UF Nascimento'].isin(estados)]['UF Nascimento'].value_counts(normalize=True).reset_index()

df_geral['proportion'] = df_geral['proportion']*100

create_bar_chart(df_geral, 'UF Nascimento', 'proportion', 
                 'Candidatos por Origem (UF): Todos os Candidatos (em %)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')




In [ ]:
df = data[['Valor Bens','UF Nascimento']]

df = df[df['UF Nascimento'].isin(estados)]

#df = df.groupby('UF Nascimento')['Valor Bens'].sum().reset_index()

df = df.sort_values('Valor Bens',ascending=False)

df.head()

create_bar_chart(df.groupby('UF Nascimento')['Valor Bens'].sum().reset_index().reset_index().sort_values('Valor Bens',ascending=False), 'UF Nascimento', 'Valor Bens', 
                'Candidatos: Valor de Bens > Valor Máximo Despesa (em R$)', 
                'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:

df_geral = dados_agg[['Valor Bens','UF Nascimento']]

df_geral = df_geral[df_geral['UF Nascimento'].isin(estados)]

#df = df.groupby('UF Nascimento')['Valor Bens'].sum().reset_index()

df_geral = df_geral.sort_values('Valor Bens',ascending=False)

create_bar_chart(df_geral.groupby('UF Nascimento')['Valor Bens'].sum().reset_index().reset_index().sort_values('Valor Bens',ascending=False), 'UF Nascimento', 'Valor Bens', 
                 'Candidatos por Origem (UF): Todos os Candidatos (em R$)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
for estado in estados:
    create_boxplot_plotly(df[df['UF Nascimento']==estado], 'UF Nascimento', 'Valor Bens', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')
    


In [ ]:
df = data['UF Nascimento'].value_counts(normalize=True).reset_index()


df = df[df['UF Nascimento'].isin(estados)]

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'UF Nascimento', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa (em %)', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg[dados_agg['Despesa de Campanha']<dados_agg['Valor Bens']]['Genêro'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'Genêro', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg[['Valor Bens','Genêro']]

create_boxplot_plotly(df, 'Genêro', 'Valor Bens', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')



In [ ]:
df = dados_agg['Genêro'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'Genêro', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg[dados_agg['Despesa de Campanha']<dados_agg['Valor Bens']]['Grau Instrução'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df, 'Grau Instrução', 'proportion', 
                 'Candidatos: Por Grau Instrução', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg['Grau Instrução'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Grau Instrução', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg[['Sigla Partido']].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

df.head()

create_bar_chart(df, 'Sigla Partido', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = dados_agg['Sigla Partido'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Sigla Partido', 'proportion', 
                 'Candidatos: Por Partido', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg[dados_agg['Despesa de Campanha']<dados_agg['Valor Bens']]['Estado Civil'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Estado Civil', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg['Estado Civil'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Estado Civil', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['Cor / Raça'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Cor / Raça', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = dados_agg['Cor / Raça'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Cor / Raça', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['Ocupação'].value_counts(normalize=True).reset_index().head(10)

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Ocupação', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['Ocupação'].value_counts(normalize=True).reset_index().head(10)

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'Ocupação', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')

In [ ]:
df = data['UF'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df, 'UF', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = data['UF'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

df = df[df['UF'].isin(estados)]

create_bar_chart(df, 'UF', 'proportion', 
                 'Candidatos: Valor de Bens > Valor Máximo Despesa', 
                 'Estado de Nascimento', 'Quantidade de Candidatos')


In [ ]:
df = data[data['Etnia Indigina'] != '#NULO#']

df = df[df['Etnia Indigina'] != 'NÃO INFORMADO']

df = df['Etnia Indigina'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100

create_bar_chart(df.head(15),
                 'Etnia Indigina',
                 'proportion',
                 'Candidatos: Valor de Bens > Valor Máximo Despesa',
                 "Etnia Indígena",
                 "Quantidade de Candidatos")


In [ ]:
df = dados_agg[dados_agg['Etnia Indigina'] != '#NULO#']

df = df[df['Etnia Indigina'] != 'NÃO INFORMADO']

df = df['Etnia Indigina'].value_counts(normalize=True).reset_index()

df['proportion'] = df['proportion']*100


create_bar_chart(df.head(10),
                 'Etnia Indigina',
                 'proportion',
                 'Candidatos: Valor de Bens > Valor Máximo Despesa',
                 "Etnia Indígena",
                 "Quantidade de Candidatos")
